# Lesson 9 - Partitioning and Bucketing

Okay, let's craft the technical notes for Lesson 9, focusing on Partitioning and Bucketing in PySpark.

---

## PySpark Technical Notes: Lesson 9 - Optimizing Data Layout: Partitioning and Bucketing

**Objective:** This section explores fundamental techniques for controlling data layout both during Spark execution (in-memory) and when storing data persistently. Proper partitioning and bucketing are critical for optimizing performance, especially for large datasets, by minimizing data shuffling and enabling efficient data pruning.

### 1. Controlling In-Memory Parallelism: `repartition` vs. `coalesce`

**Theory:**

In Spark, DataFrames (and RDDs) are divided into logical chunks called **partitions**. These partitions are the basic units of parallelism; Spark executes tasks, one per partition, concurrently across the cluster's executors. The number of partitions influences resource utilization, shuffle performance, and overall job execution time.

By default, Spark determines the number of partitions based on the input source (e.g., number of HDFS blocks) or configuration settings (`spark.sql.shuffle.partitions`, `spark.default.parallelism`). However, there are often scenarios where manually adjusting the number of partitions using transformations is beneficial. PySpark provides two primary methods for this: `repartition()` and `coalesce()`.

*   **`repartition(numPartitions, *cols)`:**
    *   **Purpose:** Increases or decreases the number of partitions. It can also redistribute data based on partitioning columns.
    *   **Mechanism:** Always triggers a **full shuffle**. Data is redistributed across the network based on either a round-robin method (if only `numPartitions` is specified) or a hash-partitioning scheme (if columns `*cols` are provided). All data is moved between executors.
    *   **Cost:** Relatively expensive due to the full shuffle (network I/O, serialization/deserialization).
    *   **Use Cases:**
        *   Increasing parallelism when the current number of partitions is too low for the available cluster resources.
        *   Distributing data more evenly to mitigate data skew before operations like joins or aggregations.
        *   Hash-partitioning data explicitly by certain keys (`*cols`) to optimize subsequent joins or group-by operations on the *same keys*, potentially avoiding future shuffles if data locality is achieved.

*   **`coalesce(numPartitions)`:**
    *   **Purpose:** Only *decreases* the number of partitions.
    *   **Mechanism:** Performs an **optimized shuffle** or avoids a shuffle altogether where possible. It tries to combine existing partitions located on the *same executor*, minimizing data movement across the network. It does *not* guarantee even data distribution across the remaining partitions.
    *   **Cost:** Significantly cheaper than `repartition` when reducing partitions, as it avoids a full shuffle.
    *   **Use Cases:**
        *   Efficiently reducing the number of partitions, typically before writing data to disk (to avoid creating too many small output files).
        *   When a drastic reduction in partitions is needed and the potential for uneven data distribution in the resulting partitions is acceptable.

**Code Example:**

```python
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Boilerplate Spark Session creation
spark = SparkSession.builder \
    .appName("RepartitionCoalesceExample") \
    .config("spark.sql.shuffle.partitions", 4) \
    .getOrCreate() # Setting shuffle partitions low for demo

# Sample Data
data = [(i, f"value_{i % 10}") for i in range(10000)]
schema = ["id", "category"]
df = spark.createDataFrame(data, schema)

# Check initial number of partitions (may depend on default parallelism)
initial_partitions = df.rdd.getNumPartitions()
print(f"Initial number of partitions: {initial_partitions}")

# Repartition to increase parallelism (causes full shuffle)
# Example: Increase partitions for potentially better processing distribution
df_repartitioned = df.repartition(10)
repartitioned_count = df_repartitioned.rdd.getNumPartitions()
print(f"Partitions after repartition(10): {repartitioned_count}")

# Repartition using hash partitioning on a column (causes full shuffle)
# Useful if subsequent operations group/join heavily on 'category'
df_hash_partitioned = df.repartition(8, "category") # 8 partitions, hashed by category
hash_partitioned_count = df_hash_partitioned.rdd.getNumPartitions()
print(f"Partitions after repartition(8, 'category'): {hash_partitioned_count}")
# Note: Observing the exact data distribution requires deeper inspection,
# but Spark will attempt to co-locate identical category values.

# Coalesce to reduce partitions (optimized shuffle, likely cheaper)
# Example: Reducing partitions before writing output
df_coalesced = df_repartitioned.coalesce(5) # Reduce from 10 down to 5
coalesced_count = df_coalesced.rdd.getNumPartitions()
print(f"Partitions after coalesce(5): {coalesced_count}")

# Trying to increase partitions with coalesce has no effect
df_coalesce_increase = df_coalesced.coalesce(10) # Trying to go from 5 to 10
coalesce_increase_count = df_coalesce_increase.rdd.getNumPartitions()
print(f"Partitions after attempting coalesce(10) from 5: {coalesce_increase_count}") # Will still be 5

spark.stop()
```

**Code Explanation:**

1.  **`spark = SparkSession.builder...getOrCreate()`**: Initializes the SparkSession. `spark.sql.shuffle.partitions` sets the default number of partitions used in shuffle operations like `groupBy`, `join`, and `repartition` (when only a number is given).
2.  **`df = spark.createDataFrame(...)`**: Creates a sample DataFrame.
3.  **`initial_partitions = df.rdd.getNumPartitions()`**: Retrieves the current number of partitions for the DataFrame's underlying RDD. The initial number depends on how the DataFrame was created and default settings.
4.  **`df_repartitioned = df.repartition(10)`**: Creates a *new* DataFrame (`df_repartitioned`) with exactly 10 partitions. This involves a full data shuffle.
5.  **`repartitioned_count = df_repartitioned.rdd.getNumPartitions()`**: Confirms the new partition count.
6.  **`df_hash_partitioned = df.repartition(8, "category")`**: Creates a new DataFrame with 8 partitions. Rows are distributed based on the hash of the `category` column value. Rows with the same category are likely to end up in the same partition. This also involves a full shuffle.
7.  **`hash_partitioned_count = df_hash_partitioned.rdd.getNumPartitions()`**: Confirms the partition count after hash partitioning.
8.  **`df_coalesced = df_repartitioned.coalesce(5)`**: Creates a *new* DataFrame by reducing the partitions of `df_repartitioned` (which had 10) down to 5. This uses the more efficient `coalesce` mechanism, minimizing network transfer.
9.  **`coalesced_count = df_coalesced.rdd.getNumPartitions()`**: Confirms the reduced partition count.
10. **`df_coalesce_increase = df_coalesced.coalesce(10)`**: Attempts to increase partitions using `coalesce`.
11. **`coalesce_increase_count = ...`**: Shows that the number of partitions remains unchanged (5), as `coalesce` cannot increase partitions.
12. **`spark.stop()`**: Releases Spark resources.

**Key Takeaway:** Use `repartition` when you need to increase partitions, guarantee a specific number of partitions with potentially better data distribution (via hash partitioning or overcoming skew), accepting the cost of a full shuffle. Use `coalesce` for a cheaper way to *reduce* the number of partitions, primarily before writing data.

---

### 2. Optimizing Data Storage and Retrieval: Disk Partitioning (`partitionBy`)

**Theory:**

Beyond controlling in-memory parallelism, Spark allows you to structure how data is stored persistently (e.g., on HDFS, S3, local filesystem). **Disk partitioning** organizes data into separate directories based on the values of one or more specified columns.

When you write a DataFrame using `partitionBy("colA", "colB", ...)`:
*   Spark creates a directory structure where each unique combination of values from the partitioning columns forms a path segment. For example: `/basePath/colA=value1/colB=valueX/`, `/basePath/colA=value1/colB=valueY/`, `/basePath/colA=value2/colB=valueX/`, etc.
*   Data files (e.g., Parquet, ORC files) containing rows that match a specific partition value combination are stored within the corresponding directory.

The primary benefit is **Predicate Pushdown** (also known as Partition Pruning). When querying data that includes filters (`WHERE` clauses) on the partitioning columns, Spark's query planner can identify the specific directories that contain the relevant data and *skip reading all other partitions*. This can dramatically reduce I/O and speed up queries, especially on very large datasets where filtering by the partition key is common.

**Considerations:**
*   **Cardinality:** Choose partitioning columns with low-to-moderate cardinality (number of unique values). Partitioning by columns with extremely high cardinality (e.g., user ID, precise timestamp) can lead to a massive number of very small directories and files, degrading filesystem performance and potentially overwhelming the Spark driver ("small files problem").
*   **File Format:** Best used with columnar formats like Parquet or ORC, which also support predicate pushdown within files.
*   **Query Patterns:** Most effective when queries frequently filter on the chosen partition keys.

**Code Example:**

```python
from pyspark.sql import SparkSession
import shutil # To clean up previous runs

# Boilerplate Spark Session creation
spark = SparkSession.builder.appName("DiskPartitioningExample").getOrCreate()

# Sample Sales Data
data = [
    (1, "2023-10-26", "Electronics", 1200.00, "North"),
    (2, "2023-10-26", "Clothing", 55.50, "South"),
    (3, "2023-10-27", "Electronics", 800.50, "North"),
    (4, "2023-10-27", "Home Goods", 250.00, "West"),
    (5, "2023-10-27", "Clothing", 80.00, "North"),
    (6, "2023-10-28", "Electronics", 150.75, "South"),
    (7, "2023-10-28", "Home Goods", 499.99, "North"),
    (8, "2023-10-28", "Clothing", 120.25, "West"),
]
schema = ["tx_id", "sale_date", "category", "amount", "region"]
sales_df = spark.createDataFrame(data, schema)

# Define the output path
output_path = "/tmp/partitioned_sales_data"

# Clean up previous output (for demonstration purposes)
try:
    shutil.rmtree(output_path)
    print(f"Removed previous directory: {output_path}")
except FileNotFoundError:
    print(f"Directory not found, proceeding: {output_path}")
except Exception as e:
    print(f"Error removing directory {output_path}: {e}")

# Write the DataFrame, partitioned by 'sale_date' and 'region'
print(f"Writing partitioned data to: {output_path}")
sales_df.write \
    .partitionBy("sale_date", "region") \
    .parquet(output_path, mode="overwrite") # Using Parquet format

print("Write complete. Check the directory structure at:", output_path)
# Expected structure (simplified):
# /tmp/partitioned_sales_data/
#   sale_date=2023-10-26/
#     region=North/
#       part-....parquet
#     region=South/
#       part-....parquet
#   sale_date=2023-10-27/
#     region=North/
#       part-....parquet
#     region=West/
#       part-....parquet
#   sale_date=2023-10-28/
#     region=North/
#       part-....parquet
#     region=South/
#       part-....parquet
#     region=West/
#       part-....parquet

# Read the partitioned data back
print("\nReading partitioned data...")
partitioned_sales_df = spark.read.parquet(output_path)

print("Schema of read data:")
partitioned_sales_df.printSchema() # Note: Partition columns are included

# Query with filter on partition columns - enables predicate pushdown
print("\nQuerying for sales on 2023-10-27 in North region...")
filtered_df = partitioned_sales_df.where(
    (F.col("sale_date") == "2023-10-27") & (F.col("region") == "North")
)

# Show the result (Spark only read data from the relevant partition directory)
filtered_df.show()

# Explain plan can often show PartitionFilters
print("\nExplain plan showing partition filters:")
filtered_df.explain()

spark.stop()
```

**Code Explanation:**

1.  **`import shutil`**: Imported to help clean up the output directory between runs of the script.
2.  **`sales_df = spark.createDataFrame(...)`**: Creates the sample DataFrame.
3.  **`output_path = "/tmp/partitioned_sales_data"`**: Defines the base directory for the output.
4.  **`shutil.rmtree(...)`**: Removes the directory if it exists from a previous run. **Use with caution!**
5.  **`sales_df.write`**: Accesses the `DataFrameWriter` API.
6.  **`.partitionBy("sale_date", "region")`**: Specifies that the output should be partitioned first by `sale_date` and then by `region`. Order matters for the directory structure.
7.  **`.parquet(output_path, mode="overwrite")`**: Writes the data in Parquet format to the specified path. `mode="overwrite"` ensures any existing data at the location is replaced.
8.  **`print("Write complete...")`**: Indicates the write operation finished and shows the expected directory layout conceptually.
9.  **`partitioned_sales_df = spark.read.parquet(output_path)`**: Reads the data back. Spark automatically detects the partitioning scheme from the directory structure and includes the partition columns (`sale_date`, `region`) in the DataFrame schema.
10. **`filtered_df = partitioned_sales_df.where(...)`**: Filters the DataFrame using the partition columns.
11. **`filtered_df.show()`**: Displays the result. Crucially, Spark only needed to scan the files within `/tmp/partitioned_sales_data/sale_date=2023-10-27/region=North/`.
12. **`filtered_df.explain()`**: Shows the physical execution plan. Look for `PartitionFilters` in the `PushedFilters` section of the scan operation (e.g., `ParquetScan`), confirming that partition pruning occurred.

---

### 3. Optimizing Joins and Aggregations: Bucketing

**Theory:**

**Bucketing** is another data layout optimization technique, often used *in conjunction with* disk partitioning (`partitionBy`) or sometimes on its own, primarily when writing data as **managed Spark tables** (stored in a metastore like Hive Metastore).

When you write a DataFrame using `bucketBy(numBuckets, colA, *cols)`:
*   Data within each disk partition (or the entire table if not partitioned) is further divided into a *fixed* number (`numBuckets`) of **buckets**.
*   Rows are assigned to a specific bucket based on the hash value of the bucketing column(s) (`colA`, `*cols`).
*   Each bucket is typically stored as a separate file within the partition directory (or table directory).
*   Optionally, data within each bucket can be sorted using `sortBy(sortColA, ...)`.

The main advantage of bucketing arises during **joins** or **aggregations** on the *bucketing keys*:
*   If two tables involved in a join are bucketed on the *same keys* using the *same number of buckets*, Spark can perform the join without a shuffle phase (a **bucket-aware merge join**). It knows that rows with the same key value will be present in the corresponding bucket number in both tables. This avoids the expensive network data transfer associated with default hash joins.
*   Similarly, aggregations (`groupBy`) on bucketing keys can be performed more efficiently within each bucket locally before a final merge.
*   Sorting within buckets (`sortBy`) can further optimize merge joins.

**Requirements & Considerations:**
*   **Managed Tables:** Bucketing information (columns, number of buckets) is typically stored in a metastore. Therefore, bucketing is primarily achieved using `saveAsTable()` rather than `write.parquet()`. An external Hive Metastore is usually required for persistence across Spark sessions.
*   **Join Keys:** Both tables in a join must be bucketed on the *exact same key(s)* and have the *exact same number of buckets* for the shuffle optimization to occur.
*   **Number of Buckets:** Choose a reasonable number. Too few buckets might not distribute data well; too many can lead to small files. Often related to the number of cores or executors.
*   **Data Skew:** Bucketing uses hashing, so severe skew in the bucketing keys can still lead to uneven bucket sizes.

**Code Example (Conceptual - Requires Hive Metastore Setup for Persistence):**

```python
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Assumes SparkSession configured with Hive support enabled
# .config("spark.sql.catalogImplementation", "hive")
# .enableHiveSupport()
spark = SparkSession.builder \
    .appName("BucketingExample") \
    .enableHiveSupport() \
    .getOrCreate()

# Sample Data for two tables to be joined
users_data = [(1, "Alice", "USA"), (2, "Bob", "CAN"), (3, "Charlie", "USA"), (4, "David", "MEX")]
users_schema = ["user_id", "name", "country"]
users_df = spark.createDataFrame(users_data, users_schema)

orders_data = [(101, 1, 50.0), (102, 3, 25.5), (103, 1, 100.0), (104, 2, 75.0), (105, 3, 10.0)]
orders_schema = ["order_id", "user_id", "amount"]
orders_df = spark.createDataFrame(orders_data, orders_schema)

# --- Save tables with Bucketing ---
# NOTE: This saves to the default Hive warehouse location unless configured otherwise.
# These tables persist across sessions if using a persistent Hive Metastore.

print("Saving users table, bucketed by user_id...")
users_df.write \
    .bucketBy(4, "user_id") \
    .sortBy("user_id") \
    .mode("overwrite") \
    .saveAsTable("users_bucketed")
print("Users table saved.")

print("Saving orders table, bucketed by user_id...")
orders_df.write \
    .bucketBy(4, "user_id") \
    .sortBy("user_id") \
    .mode("overwrite") \
    .saveAsTable("orders_bucketed")
print("Orders table saved.")


# --- Perform Join on Bucketed Tables ---
print("\nReading bucketed tables...")
users_bucketed_tbl = spark.table("users_bucketed")
orders_bucketed_tbl = spark.table("orders_bucketed")

print("\nJoining bucketed tables on user_id...")
# Spark's optimizer *should* detect the compatible bucketing and
# potentially avoid a full shuffle for this join.
joined_df = users_bucketed_tbl.join(
    orders_bucketed_tbl,
    users_bucketed_tbl["user_id"] == orders_bucketed_tbl["user_id"],
    "inner"
)

print("Showing joined results:")
joined_df.show()

# Examining the explain plan might show a SortMergeJoin without Exchange (shuffle)
# if the optimization is successful. The exact plan depends on Spark version & config.
print("\nExplain plan for the join:")
joined_df.explain()

# Clean up tables (optional)
# spark.sql("DROP TABLE IF EXISTS users_bucketed")
# spark.sql("DROP TABLE IF EXISTS orders_bucketed")

spark.stop()
```

**Code Explanation:**

1.  **`.enableHiveSupport()`**: Crucial for enabling interaction with a Hive Metastore, which is needed to store bucketing metadata persistently for `saveAsTable`.
2.  **`users_df`, `orders_df`**: Sample DataFrames created.
3.  **`users_df.write.bucketBy(4, "user_id")`**: Specifies writing the `users_df` into 4 buckets based on the hash of the `user_id` column.
4.  **`.sortBy("user_id")`**: Sorts the data within each bucket by `user_id`. This helps optimize merge joins.
5.  **`.mode("overwrite").saveAsTable("users_bucketed")`**: Saves the DataFrame as a managed table named `users_bucketed`. Bucketing and sorting metadata are stored in the metastore.
6.  **`orders_df.write...saveAsTable("orders_bucketed")`**: Saves the `orders_df` similarly, crucially using the **same number of buckets (4)** and the **same bucketing column (`user_id`)**.
7.  **`spark.table("users_bucketed")`**: Reads the data back from the managed table. Spark is aware of the bucketing structure.
8.  **`joined_df = users_bucketed_tbl.join(...)`**: Performs an inner join on `user_id`. Because both tables are compatibly bucketed on the join key, Spark's Catalyst optimizer *may* choose a shuffle-free join strategy (like SortMergeJoin directly on the buckets).
9.  **`joined_df.explain()`**: Displays the query plan. In an optimized scenario, you would *not* see an `Exchange` (shuffle) operator immediately preceding the join node for both sides of the join related to the join keys. You might see a `SortMergeJoin` operator indicating the merge strategy. *Note: Actual optimization depends on data size, Spark version, and configuration.*

---

### 4. Partitioning and Bucketing Best Practices

*   **Understand Your Data and Queries:** The most effective strategy depends heavily on data characteristics (size, distribution, cardinality) and common query patterns (filters, joins, aggregations).
*   **In-Memory Partitioning (`repartition`/`coalesce`):**
    *   Monitor task durations and executor utilization in the Spark UI. If tasks are highly uneven or parallelism is low, consider `repartition`.
    *   Use `repartition(col)` to co-locate data before joins/aggregations *if* subsequent stages benefit and the shuffle cost is acceptable.
    *   Use `coalesce()` primarily to reduce partitions efficiently before `write` operations to control the number/size of output files.
    *   Leverage **Adaptive Query Execution (AQE)** (enabled by default in recent Spark versions). AQE can dynamically coalesce shuffle partitions (`spark.sql.adaptive.coalescePartitions.enabled`) and handle skew (`spark.sql.adaptive.skewJoin.enabled`), potentially reducing the need for manual `repartition`/`coalesce` in some cases.
*   **Disk Partitioning (`partitionBy`):**
    *   Choose partitioning columns that are **frequently used in `WHERE` clauses**.
    *   Prefer columns with **low-to-moderate cardinality**. Dates, regions, categories are often good candidates. Avoid high-cardinality keys like user IDs or timestamps unless filtering on ranges is the primary goal and performance is acceptable.
    *   The order of partition columns matters for the directory structure. Place lower-cardinality columns earlier if it makes sense for typical query patterns.
    *   Be mindful of the **"small files problem"**. Excessive partitioning can create too many small files, stressing the filesystem namenode (HDFS) and potentially slowing down reads. Aim for partition file sizes in the optimal range for your filesystem (e.g., close to the HDFS block size, often 128MB-1GB). Sometimes, `coalesce` might be needed *before* writing partitioned data if the in-memory partitions are too numerous/small.
*   **Bucketing (`bucketBy`/`saveAsTable`):**
    *   Use primarily for **large tables frequently joined or aggregated** on the *same specific keys*.
    *   Requires saving as **managed tables** with Hive support enabled.
    *   Ensure the **number of buckets** and **bucketing keys** match *exactly* for tables involved in joins to enable shuffle avoidance.
    *   Consider using `sortBy` within buckets on the join/aggregation keys for further optimization (enables merge joins/aggregations).
    *   Test and verify join performance improvements via `explain()` and Spark UI analysis. Bucketing adds overhead during writes, so ensure the read/join benefits outweigh this cost.

**Conclusion:** Intelligently controlling data layout through `repartition`, `coalesce`, `partitionBy`, and `bucketBy` is a cornerstone of PySpark performance tuning. By understanding how these mechanisms work and applying them based on data and query characteristics, developers can significantly reduce shuffle overhead, minimize I/O, and accelerate Spark job execution times.

---
**End of Lesson 9 Notes**